In [3]:
import math

In [4]:
def fast_exp(base, exponent, modulus):
  result = 1
  while exponent != 0:
    if exponent % 2 == 1:
      result = (result * base) % modulus
    base = base**2 % modulus
    exponent = exponent // 2
  return result

def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def crt(remainders, moduli):
  assert len(remainders) == len(moduli), ValueError('the lists remainders and moduli are not the same length')
  assert len(remainders) > 0, ValueError('the list lengths must be greater than one')

  first_modulus = moduli[0]
  first_remainder = remainders[0]

  if len(remainders) == 1:
    return first_remainder % first_modulus, first_modulus
  
  consecutive_remainder, consecutive_modulus = crt(remainders[1:], moduli[1:])
  u, v, g = extended_euclidean(consecutive_modulus, first_modulus)

  assert g == 1, ValueError('the moduli are not relatively prime')

  return (first_remainder*consecutive_modulus*u + consecutive_remainder*first_modulus*v) % (first_modulus*consecutive_modulus), first_modulus*consecutive_modulus

def polynomial_root(exponent, a, modulus):
  u = inv_mod(exponent, modulus - 1)

  result = fast_exp(a, u, modulus)

  return result

def cf_float(x, n):
  cf = []
  for _ in range(n):
    cf.append(int(x))
    x = 1 / (x % 1)
  return cf

def continued_frac(num, den):
  cf = []
  while den != 0:
    cf.append(num//den)
    num, den = den, num % den
  return cf

def get_convergents(cf):
  convergents = [[0, 1], [1, 0]]
  for a in cf:
    hk2, hk1 = convergents[-2:]
    convergents.append([a*hk1[0] + hk2[0], a*hk1[1] + hk2[1]])
  return convergents[2:]

def crack_rsa(e, n):
  convergents = get_convergents(continued_frac(e, n))
  for convergent in convergents:
    k, d = convergent
    if k != 0 and (phi := (d * e - 1) // k) != 0:
      p_q = n - phi + 1
      disc = p_q**2 - 4*n
      if disc > 0:
        p, q = (p_q + math.isqrt(disc)) // 2, (p_q - math.isqrt(disc)) // 2
        if p*q == n:
          return d, p, q
  return -1, -1, -1

def pollard_rho(n):
  f = lambda x_: x_**2 + 1
  x = 2
  y = 2
  g = 1
  while g == 1:
    x = f(x) % n
    y = f(f(y)) % n
    _, _, g = extended_euclidean(abs(x-y), n)
  return g

### 1. Compute the simple continued fraction for the following rational numbers by hand.

$\dfrac{101}{23}$, $\dfrac{211}{199}$, and $\dfrac{239}{37}$

$\dfrac{101}{23} = 4 + \dfrac{9}{23} = 4 + \dfrac{1}{\dfrac{23}{9}} = 4 + \dfrac{1}{2 + \dfrac{5}{9}} = 4 + \dfrac{1}{2 + \dfrac{1}{\dfrac{9}{5}}} = 4 + \dfrac{1}{2 + \dfrac{1}{1 + \dfrac{4}{5}}} = 4 + \dfrac{1}{2 + \dfrac{1}{1 + \dfrac{1}{\dfrac{5}{4}}}} = 4 + \dfrac{1}{2 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{4}}}}$

$\dfrac{211}{199} = 1 + \dfrac{12}{199} = 1 + \dfrac{1}{\dfrac{199}{12}} = 1 + \dfrac{1}{16 + \dfrac{7}{12}} = 1 + \dfrac{1}{16 + \dfrac{1}{\dfrac{12}{7}}} = 1 + \dfrac{1}{16 + \dfrac{1}{1 + \dfrac{5}{7}}} = 1 + \dfrac{1}{16 + \dfrac{1}{1 + \dfrac{1}{\dfrac{7}{5}}}} = 1 + \dfrac{1}{16 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{2}{5}}}} = 1 + \dfrac{1}{16 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{\dfrac{5}{2}}}}} = 1 + \dfrac{1}{16 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{2 + \dfrac{1}{2}}}}}$

$\dfrac{239}{37} = 6 + \dfrac{17}{37} = 6 + \dfrac{1}{\dfrac{37}{17}} = 6 + \dfrac{1}{2 + \dfrac{3}{17}} = 6 + \dfrac{1}{2 + \dfrac{1}{\dfrac{17}{3}}} = 6 + \dfrac{1}{2 + \dfrac{1}{5 + \dfrac{2}{3}}} = 6 + \dfrac{1}{2 + \dfrac{1}{5 + \dfrac{1}{\dfrac{3}{2}}}} = 6 + \dfrac{1}{2 + \dfrac{1}{5 + \dfrac{1}{1 + \dfrac{1}{2}}}}$

In [12]:
cf1 = continued_frac(101,23)
cf2 = continued_frac(211,199)
cf3 = continued_frac(239,37)
print(f'{cf1=}\n{cf2=}\n{cf3=}')

cf1=[4, 2, 1, 1, 4]
cf2=[1, 16, 1, 1, 2, 2]
cf3=[6, 2, 5, 1, 2]


### 2. Using a pocket calculator (or equivalent), find the first 5 terms in the continued fraction expansion of the following numbers.

$\sqrt{7}$, $\ln{10}$, and $\sqrt[3]{19}$

$\sqrt{7} = 2.6457513110645907 = 2 + 0.6457513110645907 = 2 + \dfrac{1}{1.5485837703548633} = 2 + \dfrac{1}{1 + 0.5485837703548633} = 2 + \dfrac{1}{1 + \dfrac{1}{1.8228756555322962}} = 2 + \dfrac{1}{1 + \dfrac{1}{1 + 0.8228756555322962}} = 2 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{1.2152504370215287}}} = 2 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{1 + 0.2152504370215287}}} = 2 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{4.645751311064623}}}} = 2 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{1 + \dfrac{1}{4 + 0.645751311064623}}}}$

$\ln{10} = 2.302585092994046 = 2 + 0.302585092994046 = 2 + \dfrac{1}{3.304855470919306} = 2 + \dfrac{1}{3 + 0.304855470919306} = 2 + \dfrac{1}{3 + \dfrac{1}{3.2802429196512466}} = 2 + \dfrac{1}{3 + \dfrac{1}{3 + 0.2802429196512466}} = 2 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{3.5683327922948713}}} = 2 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{3 + 0.5683327922948713}}} = 2 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{1.759532466817724}}}} = 2 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{3 + \dfrac{1}{1 + 0.759532466817724}}}}$

$\sqrt[3]{19} = 2.668401648721945 = 2 + 0.668401648721945 = 2 + \dfrac{1}{1.496106423304171} = 2 + \dfrac{1}{1 + 0.496106423304171} = 2 + \dfrac{1}{1 + \dfrac{1}{2.0156965381334793}} = 2 + \dfrac{1}{1 + \dfrac{1}{2 + 0.0156965381334793}} = 2 + \dfrac{1}{1 + \dfrac{1}{2 + \dfrac{1}{63.70831526647842}}} = 2 + \dfrac{1}{1 + \dfrac{1}{2 + \dfrac{1}{63 + 0.70831526647842}}} = 2 + \dfrac{1}{1 + \dfrac{1}{2 + \dfrac{1}{63 + \dfrac{1}{1.411800715480508}}}} = 2 + \dfrac{1}{1 + \dfrac{1}{2 + \dfrac{1}{63 + \dfrac{1}{1 + 0.411800715480508}}}}$

In [32]:
cf1 = cf_float(7**(1/2), 5)
cf2 = cf_float(math.log(10), 5)
cf3 = cf_float(19**(1/3), 5)
print(f'{cf1=}\n{cf2=}\n{cf3=}')

cf1=[2, 1, 1, 1, 4]
cf2=[2, 3, 3, 3, 1]
cf3=[2, 1, 2, 63, 1]


### 3. For each of the following continued fractions, compute the convergents by hand using the recursive formula. You must show all of your work.

$[5, 3, 3, 2]$, $[2, 6, 4, 7]$, and $[1, 11, 2, 19]$

$[5, 3, 3, 2] = 5.304347826086956$

$h_n = 0$ $1$ $5$ $16$ $53$ $122$

$k_n = 1$ $0$ $1$ $3$ $10$ $23$

$[2, 6, 4, 7] = 2.160220994475138$

$h_n = 0$ $1$ $2$ $13$ $54$ $391$

$k_n = 1$ $0$ $1$ $6$ $25$ $181$

$[1, 11, 2, 19] = 1.0870535714285714$

$h_n = 0$ $1$ $1$ $12$ $25$ $487$

$k_n = 1$ $0$ $1$ $11$ $23$ $448$

In [40]:
conv1 = get_convergents([5, 3, 3, 2])
conv2 = get_convergents([2, 6, 4, 7])
conv3 = get_convergents([1, 11, 2, 19])
print(f'{conv1=}\n{conv2=}\n{conv3=}')

conv1=[[5, 1], [16, 3], [53, 10], [122, 23]]
conv2=[[2, 1], [13, 6], [54, 25], [391, 181]]
conv3=[[1, 1], [12, 11], [25, 23], [487, 448]]


### 4. Suppose that the decryption exponent is "small" for the following public keys (e, n). Using Python, factor n and determine d.

$(695681597973372011256194238743989992449, 1828982866109630997434176303059685778909)$ 

$(341977734951772747161616943172846708542819213047574000258429, 379557705825593928168388035830440307401877224401739990998883)$

$(8948060479536048206783929596637779074781180560054457364227047485028538848863628510727835837798064334960250020845262018976616264677278645373037345892970314930711610139327906730318428954370438683578787, 11677139426710329634259991037756069254842596788162026522091171917108309026763269757278386268596833563827715532271034008195091674722942209996173813044864736729141617204997070936964954581945163057696561)$

In [46]:
d1, p1, q1 = crack_rsa(695681597973372011256194238743989992449, 1828982866109630997434176303059685778909)
d2, p2, q2 = crack_rsa(341977734951772747161616943172846708542819213047574000258429, 379557705825593928168388035830440307401877224401739990998883)
d3, p3, q3 = crack_rsa(8948060479536048206783929596637779074781180560054457364227047485028538848863628510727835837798064334960250020845262018976616264677278645373037345892970314930711610139327906730318428954370438683578787, 11677139426710329634259991037756069254842596788162026522091171917108309026763269757278386268596833563827715532271034008195091674722942209996173813044864736729141617204997070936964954581945163057696561)
print(f'{d1=}, {p1=}, {q1=}')
print(f'{d2=}, {p2=}, {q2=}')
print(f'{d3=}, {p3=}, {q3=}')

d1=65537, p1=66405897020462343733, q1=27542476619900900873
d2=101, p2=671998030559713968361666935769, q2=564819669946735512444543556507
d3=1451730470513778492236629598992166035067, p3=5628290459057877291809182450381238927697314822133923421169378062922140081498734424133112032854812293, q3=2074722246773485207821695222107608587480996474721117292752992589912196684750549658310084416732550077


### 5. Search online to find a list of primes. Select two large primes, p and q, and a small encryption exponent d (which must be relatively prime to (p-1)(q-1)). Use the extended Euclidean algorithm to compute e. Explain the process of recovering d, p, and q.

We know,

$de = 1 (\operatorname{mod} \phi(n))$

$(m^e)^d \equiv m (\operatorname{mod} n)$

Given the chosen numbers satisfy $\left|\dfrac{k}{d}-\dfrac{e}{n}\right|<\dfrac{1}{3d^2}$ where $ed = 1 + k\phi(n)$. We can use a continuous fraction attack to approximate $\dfrac{k}{d}$ using $\dfrac{e}{n}$ since we always know $e$ and $n$. The continuous fraction attack creates the $\dfrac{e}{n}$ continuous fraction, generates the convergents, and compares if one of the convergents is equal to $\dfrac{k}{d}$.

In [59]:
# I can't find larger primes
p = 961748941
q = 982451653
d = 311

phi_n = (p-1)*(q-1)
n = p*q
# inv_mod uses extended_euclidean
e = inv_mod(d, phi_n)
_, _, g = extended_euclidean(d, phi_n)
cracked_d, cracked_p, cracked_q = crack_rsa(e, n)

print(f'Is d relatively prime to n? {g == 1}')
print(f'{e=}')
print(f'{cracked_d=}, {cracked_p=}, {cracked_q=}')

Is d relatively prime to n? True
e=45572596539175991
cracked_d=311, cracked_p=982451653, cracked_q=961748941
